In [ ]:
#once I was able to clean my initial data set I found I had access to ingredient data about the beers in
#the API so I recreated the data set with more informaiton.
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
rows = []

def parse_beer(beer):
    breweries = beer.get('breweries', [{}])
    brewery = breweries[0]
    locations = brewery.get('locations', [{}])
    location = locations[0]
    beers = [beer.get('id'),
             beer.get('name'),
             beer.get('abv'),
             beer.get('ibu'),
             beer.get('originalGravity'),
             beer.get('style',{}).get('ogMin'),
             beer.get('isOrganic'),
             beer.get('style', {}).get('categoryId'),
             beer.get('style', {}).get('category', {}).get('name'),
             beer.get('styleId'),
             beer.get('style', {}).get('name'),
             beer.get('style', {}).get('description'),
             beer.get('servingTemperatureDisplay'),
             beer.get('foodPairings'),
             beer.get('description'),
             beer.get('labels', {}).get('medium'),
             beer.get('glasswareId'),
             beer.get('glass', {}).get('name'),
             beer.get('srmId'),
             beer.get('srm', {}).get('hex'),
             beer.get('style', {}).get('srmMin'),
             beer.get('style', {}).get('srmMax'),            
             brewery.get('id'),
             brewery.get('name'),
             brewery.get('description'),
             brewery.get('established'),
             location.get('locality'),
             location.get('country', {}).get('numberCode')]

    return beers
        


pages = range(1,1182)
for page in pages:
    r = requests.get('http://api.brewerydb.com/v2/beers/?key=209db383ed1d39ec3aa1c13f89161c25&p='+str(page)+'&withBreweries=Y&withIngredients=Y')
    response = r.json()
    rows += [parse_beer(r) for r in response['data']]
    

In [ ]:
beer = pd.DataFrame(rows)

In [ ]:
#59,000 beers! 
beer.shape

In [ ]:
beer.head(3)

In [ ]:
#sadly there was still a lot of missing information.
beer.isnull().sum()

In [ ]:
beer.columns = ['id',
                'name',
                'abv',
                'ibu',
                'originalGravity',
                'ogMin',
                'isOrganic',
                'categoryId',
                'category_name',
                'styleId',
                'style_name',
                'style_description',
                'servingTemperatureDisplay',
                'foodPairings',
                'description',
                'labels',
                'glasswareId',
                'glass_name',
                'srmId',
                'srm_hex',
                'srmMin',
                'srmMax',
                'brewery_id',
                'brewery_name',
                'brewery_description',
                'brewery_established',
                'brewery_locality',
                'loc_country_code']

In [ ]:
beer.to_csv('beers_full.csv', encoding = 'utf-8')

In [ ]:
beer = pd.read_csv('beers_full.csv')

In [ ]:
beer.head(3)

In [ ]:
#now get hop, malt, and malt characteristic inforamtion from the API for the beers that have it available.
for n in range(1,1183):
    print n
    r = requests.get('http://api.brewerydb.com/v2/beers/?key=209db383ed1d39ec3aa1c13f89161c25&p='+str(n)+'&withBreweries=Y&withIngredients=Y')
    data = r.json().get('data')
    for i in range(0,len(data)):
        d = data[i]
        ingredients = d.get('ingredients', {})
        
        if len(ingredients) > 0:
            hops = ingredients.get('hops', [{}])
            for n in range(0,len(hops)):
                hop = hops[n]
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_name'] = hop.get('name')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_id'] = hop.get('id')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_alphaAcidMin'] = hop.get('alphaAcidMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_betaAcidMax'] = hop.get('betaAcidMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_betaAcidMin'] = hop.get('betaAcidMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_caryophylleneMax'] = hop.get('caryophylleneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_caryophylleneMin'] = hop.get('caryophylleneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_cohumuloneMax'] = hop.get('cohumuloneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_cohumuloneMin'] = hop.get('cohumuloneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_description'] = hop.get('description')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_farneseneMax'] = hop.get('farneseneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_humuleneMax'] = hop.get('humuleneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_humuleneMin'] = hop.get('humuleneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_myrceneMax'] = hop.get('myrceneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_myrceneMin'] = hop.get('myrceneMin')
            malts = ingredients.get('malt', [{}])
            for m in range(0,len(malts)):
                malt = malts[m]
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_name'] = malt.get('name')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_id'] = malt.get('id')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_description'] = malt.get('description')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_coarseFineDifference'] = malt.get('coarseFineDifference')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_country_code'] = malt.get('country_code')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_dryYield'] = malt.get('dryYield')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_moistureContent'] = malt.get('moistureContent')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_maxInBatch'] = malt.get('maxInBatch')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_potential'] = malt.get('potential')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_protien'] = malt.get('protien')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_srmPrecise'] = malt.get('srmPrecise')
            for m in range(0,len(malts)):
                malt = malts[m]
                chars = malt.get('characteristics', [{}])
                for l in range(0,len(chars)):
                    char = chars[l]
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_name'] = char.get('name')
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_id'] = char.get('id')
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_description'] = char.get('description')

In [ ]:
#check what we got.
beer.head(3)


In [ ]:
beer.to_csv('beers_full2.csv', encoding = 'utf-8')

In [ ]:
#create a dictionary of all the hop information for easy reference.
rows = {'hop_name': [],
        'hop_id': [],
        'hop_alphaAcidMin':[],
        'hop_betaAcidMax': [],
        'hop_betaAcidMin': [],
        'hop_caryophylleneMax': [],
        'hop_caryophylleneMin': [],
        'hop_cohumuloneMax': [],
        'hop_cohumuloneMin': [],
        'hop_description': [],
        'hop_farneseneMax': [],
        'hop_humuleneMax': [],
        'hop_humuleneMin': [],
        'hop_myrceneMax': [],
        'hop_myrceneMin': []}


for index, row in beer.iterrows():
    for n in range(0,13):
        if row['hop'+str(n)+'_name'] > 0:
            print row['hop'+str(n)+'_name']
            rows['hop_name'].append(row['hop'+str(n)+'_name'])
            rows['hop_id'].append(row['hop'+str(n)+'_id'])
            rows['hop_alphaAcidMin'].append(row['hop'+str(n)+'_alphaAcidMin'])
            rows['hop_betaAcidMax'].append(row['hop'+str(n)+'_betaAcidMax'])
            rows['hop_betaAcidMin'].append(row['hop'+str(n)+'_betaAcidMin'])
            rows['hop_caryophylleneMax'].append(row['hop'+str(n)+'_caryophylleneMax'])
            rows['hop_caryophylleneMin'].append(row['hop'+str(n)+'_caryophylleneMin'])
            rows['hop_cohumuloneMax'].append(row['hop'+str(n)+'_cohumuloneMax'])
            rows['hop_cohumuloneMin'].append(row['hop'+str(n)+'_cohumuloneMin'])
            rows['hop_description'].append(row['hop'+str(n)+'_description'])
            rows['hop_farneseneMax'].append(row['hop'+str(n)+'_farneseneMax'])
            rows['hop_humuleneMax'].append(row['hop'+str(n)+'_humuleneMax'])
            rows['hop_humuleneMin'].append(row['hop'+str(n)+'_humuleneMin'])
            rows['hop_myrceneMax'].append(row['hop'+str(n)+'_myrceneMax'])
            rows['hop_myrceneMin'].append(row['hop'+str(n)+'_myrceneMin'])
        



In [ ]:
hop_list = pd.DataFrame(rows)

In [ ]:
hop_list = hop_list.drop_duplicates(subset = 'hop_id')

In [ ]:
#now have a data set of all the hop information with no duplicates.
hop_list.to_csv('hop_list.csv', encoding = 'utf-8')

In [ ]:
#do the same for malt information
rows2 = {'malt_name': [],
        'malt_id': [],
        'malt_description':[],
        'malt_coarseFineDifference': [],
        'malt_country_code': [],
        'malt_dryYield': [],
        'malt_moistureContent': [],
        'malt_maxInBatch': [],
        'malt_potential': [],
        'malt_protien': [],
        'malt_srmPrecise': [],
       }


for index, row in beer.iterrows():
    for n in range(0,10):
        if row['malt'+str(n)+'_name'] > 0:
            print row['malt'+str(n)+'_name']
            rows2['malt_name'].append(row['malt'+str(n)+'_name'])
            rows2['malt_id'].append(row['malt'+str(n)+'_id'])
            rows2['malt_description'].append(row['malt'+str(n)+'_description'])
            rows2['malt_coarseFineDifference'].append(row['malt'+str(n)+'_coarseFineDifference'])
            rows2['malt_country_code'].append(row['malt'+str(n)+'_country_code'])
            rows2['malt_dryYield'].append(row['malt'+str(n)+'_dryYield'])
            rows2['malt_moistureContent'].append(row['malt'+str(n)+'_moistureContent'])
            rows2['malt_maxInBatch'].append(row['malt'+str(n)+'_maxInBatch'])
            rows2['malt_potential'].append(row['malt'+str(n)+'_potential'])
            rows2['malt_protien'].append(row['malt'+str(n)+'_protien'])
            rows2['malt_srmPrecise'].append(row['malt'+str(n)+'_srmPrecise'])
            
            

In [ ]:
malt_list = pd.DataFrame(rows2)


In [ ]:
malt_list = malt_list.drop_duplicates(subset = 'malt_id')

In [ ]:
malt_list.to_csv('malt_list.csv', encoding = 'utf-8')

In [ ]:
#and the same with malt characteristics.
rows3 = {'malt_name': [],
         'malt_id': [],
         'malt_char_name': [],
         'malt_char_id': [],
         'malt_char_description': []
        }
    

for index, row in beer.iterrows():
    for n in range(0,10):
        if row['malt'+str(n)+'_char0_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char0_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char0_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char0_description'])
        elif row['malt'+str(n)+'_char1_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char1_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char1_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char1_description'])
        elif row['malt'+str(n)+'_char2_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char2_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char2_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char2_description'])
        elif row['malt'+str(n)+'_char3_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char3_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char3_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char3_description'])
    for m in range(0,7):
        if row['malt'+str(m)+'_char4_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(m)+'_name'])
            rows3['malt_id'].append(row['malt'+str(m)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(m)+'_char4_name'])
            rows3['malt_char_id'].append(row['malt'+str(m)+'_char4_id'])
            rows3['malt_char_description'].append(row['malt'+str(m)+'_char4_description'])


In [ ]:
char_list = pd.DataFrame(rows3)

In [ ]:
char_list = char_list.drop_duplicates(subset = 'malt_id')

In [ ]:
char_list.to_csv('char_list.csv', encoding = 'utf-8')